In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length,udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer,VectorAssembler
from pyspark.ml import pipeline,Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator


In [0]:
SS= SparkSession.builder.appName('NLP').getOrCreate()

In [0]:
df = SS.read.csv('/FileStore/tables/SMSSpamCollection',inferSchema=True,sep='\t')

In [0]:
df.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [0]:
df= df.withColumnRenamed('_c0','Class').withColumnRenamed('_c1','text')

In [0]:
def length(x):
  return len(x)

new_udf = udf(length,IntegerType())

In [0]:
df = df.withColumn('length',new_udf(df['text']))

In [0]:
train,test = df.randomSplit([0.7,0.3])

In [0]:
# Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer
Categorize = StringIndexer(inputCol='Class',outputCol='label')
Token = Tokenizer(inputCol='text',outputCol='t_text')
Stopwords = StopWordsRemover(inputCol='t_text',outputCol='Stopwords')
countvec = CountVectorizer(inputCol='Stopwords',outputCol='Vector')
idf = IDF(inputCol='Vector',outputCol='idf_vec')
VA = VectorAssembler(inputCols=['idf_vec','length'],outputCol='features')
NB = NaiveBayes()
pipe = Pipeline(stages=[Categorize,Token,Stopwords,countvec,idf,VA,NB])

In [0]:
pipeline_p = pipe.fit(train)

In [0]:
pred = pipeline_p.transform(test)

In [0]:
mce = MulticlassClassificationEvaluator(metricName='f1')

In [0]:
eval_1 = mce.evaluate(pred)

In [0]:
eval_1

Out[71]: 0.9758783033865025

In [0]:
Bc = BinaryClassificationEvaluator(rawPredictionCol='prediction',metricName='accuracy')

In [0]:
eval_1 = mce.evaluate(pred)